# Deep MNIST for Experts

## MNIST 다운로드

In [24]:
import __future__
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./samples/MNIST_data', one_hot=True)

Extracting ./samples/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./samples/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./samples/MNIST_data/t10k-labels-idx1-ubyte.gz


## TensorFlow 대화형 세션 (InteractiveSession)

TensorFlow는 계산을 하기 위해 아주 효율적인 C ++ 백엔드에 의존합니다. 이 백엔드로의 연결을 세션(session)이라고 합니다. TensorFlow 프로그램들의 일반적인 사용법은 먼저 그래프를 만든 후 세션 안에서 그걸 실행하는 것입니다.

여기서 우리는 간편한 InteractiveSession 클래스를 대신 사용할 것인데, 이 클래스는 당신이 TensorFlow로 코드를 설계하는 방법을 훨씬 유연하게 만들어 줍니다. 계산 그래프(Computation Graph)와 그래프를 실행하는 작업들 사이에 당신이 끼어 들어갈 수 있게 해 줍니다. 이것은 IPython 등과 같은 상호작용하는 상황에서 일할때 특히 유용합니다. 여러분이 InteractiveSession을 사용하지 않는 경우에는 세션을 시작하고 그래프를 실행하기 전에 반드시 전체 계산 그래프를 모두 만들어야 합니다.

In [25]:
import tensorflow as tf
sess = tf.InteractiveSession()

placeholder: 입력 값의 형태를 미리 만들어 둠.


x는 부정소수들로 된 2d 텐서로 구성될 것입니다. 여기서 우리는 [None, 784] 의 형태를 할당했는데, 784 는 평탄화된 MNIST 이미지 하나의 차원이고, 첫번째 차원인 None은 배치 크기에 해당되는데 어떤 크기도 될 수 있다는 의미입니다. 목표 출력 클래스 y_ 또한 2d 텐서인데, 각 행은 대응되는 MNIST 이미지가 어떤 숫자 클래스에 속하는지를 나타내는 one-hot 10차원 벡터입니다.

In [26]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

가중치와 bias

Variable은 TensorFlow의 상호작용하는 작업 그래프들간에 유지되는 변경 가능한 텐서입니다. 계산 과정에서 사용되거나 심지어 변경될 수도 있습니다. 기계학습 응용 사례들을 위해 일반적으로 모델 파라미터 Variables 를 사용합니다.

In [27]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

우리는 각 파라미터의 초기값을 tf.Variable에 주었습니다. 이 경우, 우리는 W와 b 둘 모두 0으로 채워진 텐서로 초기화했습니다. W는 784x10 행렬 (우리가 784개의 입력 특징(이미지의 픽셀수)과 10개의 출력이 있으므로) 이며, b는 10차원 벡터입니다. (10개의 클래스가 있으니까요)

Variables가 세션 안에서 사용되기 전에 반드시 그 세션을 사용하여 초기화되어야 합니다. 이 단계는 이미 지정되어 있는 초기값을 (이 경우 0으로 채워진 텐서) 사용하고, 각 Variable에 할당해야 합니다. 이 과정은 모든 Variables 에 대하여 한 번에 수행할 수 있습니다.

In [28]:
sess.run(tf.initialize_all_variables())

softmax: 벡터화된 입력 이미지 x를 가중치 행렬 W로 곱하고, 편향(bias) b를 더한 다음 각 클래스에 지정된 소프트맥스 확률들을 계산합니다.

In [29]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

## cost function: cross entropy로 설정

In [30]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

## Training: Gradient Descent

In [31]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [32]:
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [33]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [34]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [35]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9092


# Build a Multilayer Convolutional Network

In [36]:
init = tf.initialize_all_variables()
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'
with tf.Session(config = config) as s:
    sess.run(init)

## Weight Initialization

모델을 만들기 위해, 많은 weight와 bias를 만들어야 할 필요가 있다. gradient가 0이 나오는 평형 상태를 방지하고 평형 상태를 깨기 위해 (symmetry breaking) 적은 양의 노이즈로 초기화해야 한다. 우리는 ReLU를 쓸 것이므로, “dead neuron” 을 피하기 위해 약간 양의 초기 bias로 초기화하는 것도 괜찮다. 이를 하기 전에 두가지 편리한 함수를 만들어두자.

In [37]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

## Convolution and Pooling

합성곱은 하나의 이동 크기를 사용하고, 0로 패딩되어 결과적으로 출력의 크기가 입력의 크기와 같게 됩니다. 우리의 풀링은 2x2 블럭의 평범한 max pooling 입니다.

In [59]:
# convolution & max pooling
# vanila version of CNN
# x (아래 함수들에서) : A 4-D `Tensor` with shape `[batch, height, width, channels]`
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

## First Convolutional Layer

input patch=5*5, input channel=1, output=32

In [39]:
# [5, 5, 1, 32]: 5x5 convolution patch, 1 input channel, 32 output channel.
# MNIST의 pixel은 0/1로 표현되는 1개의 벡터이므로 1 input channel임.
# Shape을 아래와 같이 넣으면 넣은 그대로 5x5x1x32의 텐서를 생성함.
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
# 최종적으로, 32개의 output channel에 대해 각각 5x5의 convolution patch (filter) weight 와 1개의 bias 를 갖게 됨.

In [40]:
# 레이어를 적용하기 위하여 우선 x를 4차원 텐서의 형태로 변환하는데, 
# 두번째와 세번째 차원을 이미지의 폭과 높이에 해당하고, 마지막 차원은 색깔 채널의 수에 대응하는 차원.
# x는 [None, 784] (위 placeholder에서 선언). 이건 [batch, 28*28] 이다.
# x_image는 [batch, 28, 28, 1] 이 됨. -1은 batch size를 유지하는 것이고 1은 color channel.
x_image = tf.reshape(x, [-1,28,28,1])

In [41]:
# 이제, x_image를 weight tensor와 convolve하고 bias를 더한 뒤 ReLU를 적용하자. 그리고 마지막으론 max pooling.
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## Second Convolutional Layer

In [42]:
# channels (features) : 32 => 64
# 5x5x32x64 짜리 weights.
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

## Densely Connected Layer

2x2의 max pooling을 두번 거쳤으니 이제 우리가 갖고있는 이미지의 크기는 7x7이다. 이제 전체 이미지에 연결된 1024개의 뉴런으로 구성된 fully-connected layer를 추가하자. 이전 레이어, 즉 풀링 레이어2 (h_pool2) 의 텐서를 batch of vector로 변환하고, weight matrix를 곱하고, bias를 더하고, ReLU를 적용하자.

In [43]:
# Densely connected layer
# 7*7*64는 h_pool2의 output (7*7의 reduced image * 64개의 채널). 1024는 fc layer의 뉴런 수.
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## Dropout

오버피팅을 줄이기 위해 우리는 판독 레이어 전에 탈락을 적용합니다. 뉴런의 출력이 탈락 동안 유지될 확률에 대한 placeholder를 만듭니다. 이건 훈련 도중에는 탈락을 설정하고 테스트 기간동안에는 해제할 수 있게 해 줍니다. TensorFlow의 tf.nn.dropout 작업은 자동으로 뉴런 출력에 마스킹을 할 뿐 아니라 스케일을 조정해 주므로, 탈락은 어떠한 추가적인 스케일링 없이 동작합니다.

In [44]:
# keep_prob은 dropout을 적용할지 말지에 대한 확률임. 이를 이용해서 training 동안만 드롭아웃을 적용하고 testing 때는 적용하지 않는다.
# training & evaluation 코드를 보니 keep_prob = 1.0일때 dropout off 인 듯.
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## Readout Layer

In [58]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## Train and Evaluate the Model

트레이닝과 평가는 Softmax Regression에서 했던 것과 동일하다. 단, optimizer를 steepest gradient descent 대신에 ADAM 을 사용한다. 또한 dropout 확률인 keep_prob가 feed_dict에 추가된다.

In [50]:
from __future__ import print_function
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())
init = tf.initialize_all_variables()
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC'
with tf.Session(config = config) as s:
    sess.run(init)
    
for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
     train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
     print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.14
step 100, training accuracy 0.82
step 200, training accuracy 0.92
step 300, training accuracy 0.76
step 400, training accuracy 0.96
step 500, training accuracy 0.94
step 600, training accuracy 0.96
step 700, training accuracy 0.96
step 800, training accuracy 0.88
step 900, training accuracy 0.96
step 1000, training accuracy 0.96
step 1100, training accuracy 1
step 1200, training accuracy 0.94
step 1300, training accuracy 1
step 1400, training accuracy 0.96
step 1500, training accuracy 0.94
step 1600, training accuracy 1
step 1700, training accuracy 1
step 1800, training accuracy 0.98
step 1900, training accuracy 0.96
step 2000, training accuracy 0.98
step 2100, training accuracy 0.94
step 2200, training accuracy 1
step 2300, training accuracy 0.96
step 2400, training accuracy 1
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 0.94
step 2800, training accuracy 0.98
step 2900, training accuracy 0.98
step 3000, train

In [57]:
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images[:512], y_: mnist.test.labels[:512], keep_prob: 1.0}))

test accuracy 0.994141
